In [1]:
import cProfile
import os
import random as rd
import time
from collections import defaultdict
import numpy as np
import itertools as it
import Utilities.experiment_utils as eu
import unit_test as ut
import Games.mnk as mnk
import Agents.random as arand
import Agents.vanilla_mcts as mcts 
import Agents.siea_mcts as siea_mcts
import Games.function_optimisation as fo
import Utilities.logs_management as lm
#cProfile.run("wins =  random_games(10000, base_gs)")

In [ ]:
function_index = 1
runs = 3
random_seed = 1
iterations = 50
game_state = fo.GameState(function_index=function_index)
game_state.set_initial_state()
mcts_player = mcts.MCTS_Player(max_iterations=iterations, logs=True)
action = eu.mcts_decision_analysis(game_state, mcts_player, os.path.join("Outputs","FO_single_decision","Function_" + str(function_index), mcts_player.name), runs, random_seed)

In [ ]:
#tested_agent.evolution_logs
#tested_agent.choose_action_logs
#print(tested_agent.view_mcts_tree())
game = mnk.GameState(10,10,5)
game.set_initial_state()
agent2 = mcts.MCTS_Player(max_fm=10000, logs=True)
agent1 = siea_mcts.SIEA_MCTS_Player(max_fm=10000, logs=True)
agent0 = arand.RandomPlayer()
game_player = eu.GamePlayer(game, [agent2, agent1])
game_player.play_games(50, logs=True)
#game_player.logs_by_action
game_player.save_data(os.path.join("Outputs","MNK", agent2.name + "vs" + agent1.name))

game_player = eu.GamePlayer(game, [agent1, agent2])
game_player.play_games(50, logs=True)
#game_player.logs_by_action
game_player.save_data(os.path.join("Outputs","MNK", agent1.name + "vs" + agent2.name))

In [ ]:
ut.run()